In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import json
from itertools import islice
import torch
import torch.nn.functional as F
from transformers import (
    PreTrainedTokenizerFast,
    AutoModelForCausalLM,
    AutoTokenizer,
    )

# Upload tools

In [3]:
# qwen_coder_model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
# qwen_coder_model = AutoModelForCausalLM.from_pretrained(
#     qwen_coder_model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )


# qwen_model_name = "Qwen/Qwen2.5-7B-Instruct"
# qwen_model = AutoModelForCausalLM.from_pretrained(
#     qwen_model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )


# athene_model_name = "Nexusflow/Athene-V2-Chat"
# athene_model = AutoModelForCausalLM.from_pretrained(athene_model_name)


# deepseek_model_name = "deepseek-ai/DeepSeek-V2-Lite"
# deepseek_model = AutoModelForCausalLM.from_pretrained(deepseek_model_name,
#                                                       trust_remote_code=True)  


# deepseek_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
# deepseek_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")


# deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1")
# deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True)  


# athene_tokenizer = AutoTokenizer.from_pretrained("Nexusflow/Athene-V2-Chat")
# athene_model = AutoModelForCausalLM.from_pretrained("Nexusflow/Athene-V2-Chat")

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
# qwen_tokenizer_path = "/kaggle/input/tokenizers/qwen_tokenizer.json"
# deepseek_tokenizer_path = "/kaggle/input/tokenizers/deepseek_tokenizer.json"
# llama_tokenizer_path = "/kaggle/input/tokenizers/llama_tokenizer.json"

# with open(qwen_tokenizer_path, 'r') as file:
#     qwen_tokenizer = PreTrainedTokenizerFast(tokenizer_file=qwen_tokenizer_path)
#     qwen = json.load(file)
#     qwen_vocab = qwen['model']['vocab']
# with open(deepseek_tokenizer_path, 'r') as file:
#     deepseek_tokenizer = PreTrainedTokenizerFast(tokenizer_file=deepseek_tokenizer_path)
#     deepseek = json.load(file)
#     deepseek_vocab = deepseek['model']['vocab']
# with open(llama_tokenizer_path, 'r') as file:
#     llama_tokenizer = PreTrainedTokenizerFast(tokenizer_file=llama_tokenizer_path)
#     llama = json.load(file)
#     llama_vocab = llama['model']['vocab']

In [4]:
python_keywords = [
    "def", "class", "import", "from", "as", "if", "elif", "else", "while", 
    "for", "in", "break", "continue", "return", "yield", "try", "except", 
    "finally", "with", "assert", "lambda", "global", "nonlocal", "pass", 
    "raise", "True", "False", "None", "is", "not"
]

java_keywords = [
    "class", "interface", "extends", "implements", "package", "import", "public", 
    "private", "protected", "static", "final", "abstract", "synchronized", 
    "volatile", "transient", "native", "strictfp", "void", "int", "double", 
    "float", "char", "boolean", "long", "short", "byte", "if", "else", "while",
    "for"
]


cpp_keywords = [
    "int", "float", "double", "char", "void", "bool", "short", "long", "signed", 
    "unsigned", "if", "else", "switch", "case", "default", "for", "while", 
    "do", "break", "continue", "return", "goto", "class", "struct", "union", 
    "namespace", "using", "private", "protected", "public"
]

go_keywords = [
    "break", "case", "chan", "const", "continue", "default", "defer", "else", 
    "fallthrough", "for", "func", "go", "goto", "if", "import", "interface", 
    "map", "package", "range", "return", "select", "struct", "switch", "type", 
    "var", "bool", "byte", "complex64", "complex128", "float32"
]

react_keywords = [
    "React", "Component", "useState", "useEffect", "useContext", "useReducer", 
    "useMemo", "useCallback", "useRef", "useLayoutEffect", "useImperativeHandle", 
    "jsx", "props", "state", "context", "provider", "consumer", "hook", 
    "fragment", "key", "ref", "render", "virtualDOM", "reconciliation", 
    "portal", "suspense", "lazy", "errorBoundary", "children", "strictMode"
]

javascript_keywords = [
    "var", "let", "const", "if", "else", "switch", "case", "default", "for", 
    "while", "do", "break", "continue", "return", "function", "class", 
    "extends", "constructor", "super", "import", "export", "try",
    "catch", "finally", "throw", "this", "new", "delete", "instanceof", "typeof"
]

rust_keywords = [
    "as", "break", "const", "continue", "crate", "else", "enum", "extern", 
    "false", "fn", "for", "if", "impl", "in", "let", "loop", "match", 
    "mod", "move", "mut", "pub", "ref", "return", "Self", "self", "static", 
    "struct", "super", "trait", "true"
]

php_keywords = [
    "abstract", "and", "array", "as", "break", "callable", "case", "catch", 
    "class", "clone", "const", "continue", "declare", "default", "do", "echo", 
    "else", "elseif", "empty", "enddeclare", "endfor", "endforeach", "endif", 
    "endswitch", "endwhile", "eval", "exit", "extends", "final", "finally"
]

all_keywords = [python_keywords,
                java_keywords,
                cpp_keywords,
                go_keywords,
                react_keywords,
                javascript_keywords,
                rust_keywords,
                php_keywords,]

# Warm start

In [5]:
def get_token_probabilities(model, tokenizer, text, top_k=None):
    """
    Get token probabilities from a GPT model for each position in the input text.

    Args:
        model: GPT model instance
        tokenizer: GPT tokenizer instance
        text: Input text to analyze
        top_k: Optional, number of top probabilities to return for each position

    Returns:
        List of dictionaries containing token probabilities for each position
    """
    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors='pt')

    # Get model's raw predictions
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    # Convert logits to probabilities using softmax
    probs = F.softmax(logits[0], dim=-1)

    # Store results for each position
    token_probs = []

    # Analyze each position
    for position in range(len(input_ids[0])):
        position_probs = probs[position]

        # Get token indices sorted by probability
        sorted_indices = torch.argsort(position_probs, descending=True)

        # Limit to top_k if specified
        if top_k:
            sorted_indices = sorted_indices[:top_k]

        # Create dictionary of token:probability pairs
        position_dict = {
            'position': position,
            'token': tokenizer.decode(input_ids[0][position]),
            'probabilities': {
                tokenizer.decode([idx.item()]): position_probs[idx].item()
                for idx in sorted_indices
            }
        }

        token_probs.append(position_dict)

    return token_probs

In [6]:
# Example usage
def analyze_text(text, model, tokenizer, top_k=50):
    """
    Analyze token probabilities for a given text using specified GPT model.

    Args:
        text: Input text to analyze
        model_name: Name of the GPT model to use
        top_k: Number of top probabilities to show for each position
    """
    # Load model and tokenizer
    # model = GPT2LMHeadModel.from_pretrained(model_name)
    # tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Get token probabilities
    results = get_token_probabilities(model, tokenizer, text, top_k)
    
    # Print results
    # print(f"Analysis of text: '{text}'\n")
    # for pos_data in results[:5]:
    #     print(f"Position {pos_data['position']}: Token '{pos_data['token']}'")
    #     print("Top probabilities:")
    #     for token, prob in islice(pos_data['probabilities'].items(), 3):
    #         print(f"->{token}<-: {prob:.4f}")
    #     print()
        
    tokens_prob = {}
    for i in range(len(results)):
        given_token = results[i]['token']
        tokens_prob[given_token] = {}
        
        for token, prob in islice(results[i]['probabilities'].items(), 3):
            tokens_prob[given_token][token] = prob

    return tokens_prob

## Check on specific programming words

In [7]:
from typing import List

def warm_start(words: List[str], model, tokenizer):
    words = ' '.join(words)
    result = analyze_text(words, model, tokenizer)

    return result

### Qwen2.5-coder

In [10]:
with open("qwen-2-5-coder.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, qwen_coder_model, qwen_tokenizer)
        results[f"language_{i}"] = warm_start_result
    
    json.dump(results, f, ensure_ascii=False, indent=4)

### Qwen-2.5

In [14]:
with open("qwen-2-5.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, qwen_model, qwen_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

### Athene

In [ ]:
with open("athene.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, athene_model, athene_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

### Deepseek r1

In [ ]:
with open("deepseek-r1.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

### DeepSeek-R1-Distill-Qwen-7B

In [ ]:
deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

with open("deepseek-r1-qwen-7b.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### DeepSeek-R1-Distill-Qwen-1.5B

In [8]:
deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

with open("deepseek-r1-qwen-1.5b.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result
    
    json.dump(results, f, ensure_ascii=False, indent=4)

### DeepSeek-R1-Distill-Llama-8B 

In [ ]:
deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

with open("deepseek-r1-llama-8b.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

# Miss words in tokenizers

In [21]:
misswords = ['nonlocal',
             'synchronized',
             'transient',
             'strictfp',
             'fallthrough',
             'complex64',
             'complex128',
             'float32',
             'useEffect',
             'useContext',
             'useReducer',
             'useMemo',
             'useCallback',
             'useRef',
             'useLayoutEffect',
             'useImperativeHandle',
             'virtualDOM',
             'reconciliation',
             'suspense',
             'errorBoundary',
             'useStrictMode',
             'instanceof',
             'enddeclare',
             'endfor',
             'endswitch',
             'endwhile',
            ]

In [35]:
import random
from termcolor import colored

colors = ['red', 'green', 'yellow', 'blue']

def tokenize_and_colorize(tokenizer, code):
    tokens = tokenizer.tokenize(code)
    colored_code = []
    last_color = None

    for tok in tokens:
        new_color = random.choice([color for color in colors if color != last_color])
        last_color = new_color
        colored_code.append(colored(tok, new_color))

    return ' '.join(colored_code)

## Qwen2

In [38]:
for missword in misswords:
    colored_code = tokenize_and_colorize(qwen_tokenizer, missword)
    print(colored_code)

non local
s ynchronized
trans ient
strict fp
fall through
complex 6 4
complex 1 2 8
float 3 2
use Effect
use Context
use Reducer
use Memo
use Callback
use Ref
use Layout Effect
use Im per ative Handle
virtual DOM
re conciliation
s usp ense
error Boundary
use Strict Mode
instance of
end declare
end for
ends witch
end while


## Deepseek r1

In [36]:
for missword in misswords:
    colored_code = tokenize_and_colorize(deepseek_tokenizer, missword)
    print(colored_code)

non local
s ynchronized
trans ient
strict fp
fall through
complex 6 4
complex 1 2 8
float 3 2
use Effect
use Context
use Reducer
use Memo
use Callback
use Ref
use Layout Effect
use Im per ative Handle
virtual DOM
re conciliation
s usp ense
error Boundary
use Strict Mode
instance of
end declare
end for
ends witch
end while


## Llama 3.1-405B

In [39]:
for missword in misswords:
    colored_code = tokenize_and_colorize(llama_tokenizer, missword)
    print(colored_code)

non local
s ynchronized
trans ient
strict fp
fall through
complex 64
complex 128
float 32
use Effect
use Context
use Reducer
use Memo
use Callback
use Ref
use Layout Effect
use Im per ative Handle
virtual DOM
re conciliation
s usp ense
error Boundary
use Strict Mode
instance of
end declare
end for
ends witch
end while
